In [1]:
import tensorflow as tf

### Display devices

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6542228448468619228
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 22654255104
locality {
  bus_id: 2
}
incarnation: 12687903592658399396
physical_device_desc: "device: 0, name: Tesla P40, pci bus id: 0000:83:00.0, compute capability: 6.1"
]


### Construct LR elements

In [3]:
# Model parameters
W = tf.Variable([.3], dtype=tf.float32) 
b = tf.Variable([-.3], dtype=tf.float32)
# Model input and output
x = tf.placeholder(tf.float32)
linear_model = W*x + b
y = tf.placeholder(tf.float32)

# loss
loss = tf.reduce_sum(tf.square(linear_model - y)) # sum of the squares

### Evaluation

To actually evaluate the nodes, we must run the computational graph within a session. A session encapsulates the control and state of the TensorFlow runtime.

In [4]:
# training loop
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init) # reset values to wrong

In [9]:
sess.run([W, b])

[array([ 0.30000001], dtype=float32), array([-0.30000001], dtype=float32)]

In [10]:
sess.run(linear_model, {x: [1., 2., 3., 4.]})

array([ 0.        ,  0.30000001,  0.60000002,  0.90000004], dtype=float32)

In [6]:
sess.run(tf.square(linear_model - y), {x: [1., 2., 3., 4.], y: [0, -1, -2, -3]})

array([  0.        ,   1.68999982,   6.75999928,  15.21000099], dtype=float32)

In [11]:
sess.run(loss, {x: [1., 2., 3., 4.], y: [0, -1, -2, -3]})

23.660002

### Training

In [13]:
# optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [16]:
# training data
x_train = [1, 2, 3, 4]
y_train = [0, -1, -2, -3]

for i in range(1000):
  sess.run(train, {x: x_train, y: y_train})

# evaluate training accuracy
curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x: x_train, y: y_train})
print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))

W: [-0.9999969] b: [ 0.99999082] loss: 5.69997e-11


## tf.estimator 

tf.estimator is a high-level TensorFlow library that simplifies the mechanics of machine learning, including the following:

- running training loops
- running evaluation loops
- managing data sets
- tf.estimator defines many **predefined models**.

In [17]:
# NumPy is often used to load, manipulate and preprocess data.
import numpy as np

# Declare list of features. We only have one numeric feature. There are many
# other types of columns that are more complicated and useful.
feature_columns = [tf.feature_column.numeric_column("x", shape=[1])]

# An estimator is the front end to invoke training (fitting) and evaluation
# (inference). There are many predefined types like linear regression,
# linear classification, and many neural network classifiers and regressors.
# The following code provides an estimator that does linear regression.
estimator = tf.estimator.LinearRegressor(feature_columns=feature_columns)

# TensorFlow provides many helper methods to read and set up data sets.
# Here we use two data sets: one for training and one for evaluation
# We have to tell the function how many batches
# of data (num_epochs) we want and how big each batch should be.
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

# We can invoke 1000 training steps by invoking the  method and passing the
# training data set.
estimator.train(input_fn=input_fn, steps=1000)

# Here we evaluate how well our model did.
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f679c1637d0>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/tmp/tmp_fHnF_', '_save_summary_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmp_fHnF_/model.ckpt.
INFO:tensorflow:loss = 13.0, step = 1
INFO:tensorflow:global_step/sec: 510.696
INFO:tensorflow:loss = 0.118706, step = 101 (0.198 sec)
INFO:tensorflow:global_step/sec: 729.007
INFO:tensorflow:loss = 0.0506629, step = 201 (0.136 sec)
INFO:tensorflow:global_step/sec: 747.071
INFO:tensorflow:loss = 0.0171695

You can also add custom model in tf.estimator